In [1]:
# !pip install konlpy

In [1]:
# -*- coding: utf-8 -*-

import time
from tqdm import tqdm
from konlpy import tag
from konlpy.corpus import kolaw
from konlpy.utils import pprint
from collections import Counter

import pandas as pd
import re

In [2]:
from konlpy.tag import Kkma
kkma = Kkma()
print(kkma.pos(u'React.js 를 활용한 개발, 운영 경험이 있으신 분'))

[('React', 'OL'), ('.', 'SF'), ('js', 'OL'), ('를', 'JKO'), ('활용', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('개발', 'NNG'), (',', 'SP'), ('운영', 'NNG'), ('경험', 'NNG'), ('이', 'JKS'), ('있', 'VV'), ('으시', 'EPH'), ('ㄴ', 'ETD'), ('분', 'NNG')]


In [3]:
#빠른 한나눔으로 테스트
from konlpy.tag import Hannanum
hannanum = Hannanum()
print(hannanum.pos(u'React.js 를 활용한 개발, 운영 경험이 있으신 분'))

[('React', 'F'), ('.', 'S'), ('js', 'F'), ('를', 'N'), ('활용', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('개발', 'N'), (',', 'S'), ('운영', 'N'), ('경험', 'N'), ('이', 'J'), ('있', 'P'), ('으시ㄴ', 'E'), ('분', 'N')]


In [4]:
from konlpy.tag import Okt
okt = Okt()
print(okt.pos(u'React.js 를 활용한 개발, 운영 경험이 있으신 분'))

[('React', 'Alpha'), ('.', 'Punctuation'), ('js', 'Alpha'), ('를', 'Noun'), ('활용', 'Noun'), ('한', 'Josa'), ('개발', 'Noun'), (',', 'Punctuation'), ('운영', 'Noun'), ('경험', 'Noun'), ('이', 'Josa'), ('있으신', 'Adjective'), ('분', 'Noun')]


In [5]:
from konlpy.tag import Komoran
komoran = Komoran()
print(komoran.pos(u'React.js 를 활용한 개발, 운영 경험이 있으신 분'))

[('React', 'SL'), ('.', 'SF'), ('js', 'SL'), ('를', 'JKO'), ('활용', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETM'), ('개발', 'NNG'), (',', 'SP'), ('운영', 'NNG'), ('경험', 'NNG'), ('이', 'JKS'), ('있', 'VX'), ('으시', 'EP'), ('ㄴ', 'ETM'), ('분', 'NNB')]


In [6]:
df = pd.read_csv('크롤링_원티드_ver02_all_230627.csv')
df.head()

,직무명,채용공고명,해시태그,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,근무지역,회사명,회사분야,출처
0,웹 개발자,Python Developer (백엔드 주니어),#퇴사율5%이하#51~300명#설립4~9년#육아휴직#출산휴가#스톡옵션#건강검진#교육...,"- 간단소개\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 FaaS...",• 세무 알고리즘 자동화 설계 및 개발\n• 세무팀과 커뮤니케이션\n• AWS 인프...,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\n• Python(필수), 웹...","• MSA 기반의 시스템 개발 경험이 있으신 분\n• AWS 를 활용한 개발, 운영...",# 몰입을 위한\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 내 ...,Github\nJavaScript\nTypeScript\nReact.js\nVue.JS,서울.한국,혜움랩스,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167436
1,웹 개발자,프론트엔드 개발자 (주니어),#퇴사율5%이하#51~300명#설립4~9년#육아휴직#출산휴가#스톡옵션#건강검진#교육...,"- 간단소개\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 FaaS...","• 세무 관련 서비스 및 플랫폼의 프론트엔드 에플리케이션 개발\n• 요구사항 분석,...",• HTML / CSS / JavaScript 에 대한 이해\n• 컴퓨터 사이언스에...,"• Vue.js, React.js, Angular 등 SPA 개발 경험이 있는 분\...",# 몰입을 위한\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 내 ...,Github\nJavaScript\nTypeScript\nReact.js\nVue.JS,서울.한국,혜움랩스,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167433
2,웹 개발자,프론트엔드 개발자 Software Engineer (3년 이상) 사본,"#출산휴가#스타트업#반려동물#음료#커피#간식#건강검진#운동비#IT, 컨텐츠",유섭은 현재 번역 서비스 (Subup) 및 자막 편집 에디터(Alt-Editor)를...,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 프론트엔드 서비스 개발 및 유지 관리 경험을 가지신...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,• 자유로운 휴가 (무제한 무급 휴가) \n• 동료와 원활한 협업을 위해 11시부터...,NaN,서울.한국,유섭,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167134
3,웹 개발자,백엔드 개발자 (Backend Software Engineer | 3년 이상) 사본,"#출산휴가#스타트업#반려동물#음료#커피#간식#건강검진#운동비#IT, 컨텐츠",유섭은 현재 번역 서비스 (Subup) 및 자막 편집 에디터(Alt-Editor)를...,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 백엔드 서비스 개발 및 유지 관리 경험을 가지신 분...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,• 자유로운 휴가 (무제한 무급 휴가) \n• 동료와 원활한 협업을 위해 11시부터...,NaN,서울.한국,유섭,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167133
4,웹 개발자,웹개발 인재(PHP),"#인원급성장#51~300명#설립10년이상#반려동물#음료#사내카페#복지포인트#IT, 컨텐츠",• 다음과 같이 유능하고 역량있는 인재를 채용합니다.\n당사와 미래를 함께 이끌어 ...,• 웹기반 검사시스템 개발 및 유지보수\n\n• 신규 프로젝트 개발,"PHP 중급 이상\nMYSQL 중급 이상\nPHP 프레임워크 사용 경험자 (ci, ...",웹 서비스 라이브 런칭 경험자\nRESTful API 설계 및 개발경험\nGit H...,매월 1회 패밀리데이(오전근무/세번째 금요일)\n패밀리데이 상품권 지급(연 60만원...,Git\nLinux\nMySQL\nPHP\nDocker\nSVN,경기.한국,인투씨엔에스,"IT, 컨텐츠",https://www.wanted.co.kr/wd/166922


In [7]:
df['직무명'].value_counts()

프론트엔드 개발자                        1130
웹 개발자                            1100
C,C++ 개발자                         534
파이썬 개발자                           449
Node.js 개발자                       390
머신러닝 엔지니어                         369
iOS 개발자                           362
자바 개발자                            340
데이터 엔지니어                          333
DevOps / 시스템 관리자                  331
시스템,네트워크 관리자                      312
안드로이드 개발자                         280
개발 매니저                            264
기술지원                              239
보안 엔지니어                           198
임베디드 개발자                          186
하드웨어 엔지니어                         154
서버 개발자                            120
크로스플랫폼 앱 개발자                      103
블록체인 플랫폼 엔지니어                     101
소프트웨어 엔지니어                        100
ERP전문가                             80
웹 퍼블리셔                             80
.NET 개발자                           76
영상,음성 엔지니어                         76
QA,테스트 엔지니어                        60
그래픽스 엔지니어   

In [8]:
# "직무명, 자격요건, 우대사항, 기술스택" col만 선택
df1 = df[["직무명","주요업무","자격요건","우대사항","기술스택 ・ 툴"]]
df1.head()

,직무명,주요업무,자격요건,우대사항,기술스택 ・ 툴
0,웹 개발자,• 세무 알고리즘 자동화 설계 및 개발\n• 세무팀과 커뮤니케이션\n• AWS 인프...,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\n• Python(필수), 웹...","• MSA 기반의 시스템 개발 경험이 있으신 분\n• AWS 를 활용한 개발, 운영...",Github\nJavaScript\nTypeScript\nReact.js\nVue.JS
1,웹 개발자,"• 세무 관련 서비스 및 플랫폼의 프론트엔드 에플리케이션 개발\n• 요구사항 분석,...",• HTML / CSS / JavaScript 에 대한 이해\n• 컴퓨터 사이언스에...,"• Vue.js, React.js, Angular 등 SPA 개발 경험이 있는 분\...",Github\nJavaScript\nTypeScript\nReact.js\nVue.JS
2,웹 개발자,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 프론트엔드 서비스 개발 및 유지 관리 경험을 가지신...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,NaN
3,웹 개발자,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 백엔드 서비스 개발 및 유지 관리 경험을 가지신 분...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,NaN
4,웹 개발자,• 웹기반 검사시스템 개발 및 유지보수\n\n• 신규 프로젝트 개발,"PHP 중급 이상\nMYSQL 중급 이상\nPHP 프레임워크 사용 경험자 (ci, ...",웹 서비스 라이브 런칭 경험자\nRESTful API 설계 및 개발경험\nGit H...,Git\nLinux\nMySQL\nPHP\nDocker\nSVN


In [9]:
# C-level 직무인 CTO, CIO 삭제
print(len(df1))
df2 = df1[df1["직무명"]!="CTO,Chief Technology Officer"]
df2 = df2[df2["직무명"]!="CIO,Chief Information Officer"]

print(len(df2))
df2.head()

8095
8045


,직무명,주요업무,자격요건,우대사항,기술스택 ・ 툴
0,웹 개발자,• 세무 알고리즘 자동화 설계 및 개발\n• 세무팀과 커뮤니케이션\n• AWS 인프...,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\n• Python(필수), 웹...","• MSA 기반의 시스템 개발 경험이 있으신 분\n• AWS 를 활용한 개발, 운영...",Github\nJavaScript\nTypeScript\nReact.js\nVue.JS
1,웹 개발자,"• 세무 관련 서비스 및 플랫폼의 프론트엔드 에플리케이션 개발\n• 요구사항 분석,...",• HTML / CSS / JavaScript 에 대한 이해\n• 컴퓨터 사이언스에...,"• Vue.js, React.js, Angular 등 SPA 개발 경험이 있는 분\...",Github\nJavaScript\nTypeScript\nReact.js\nVue.JS
2,웹 개발자,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 프론트엔드 서비스 개발 및 유지 관리 경험을 가지신...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,NaN
3,웹 개발자,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 백엔드 서비스 개발 및 유지 관리 경험을 가지신 분...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,NaN
4,웹 개발자,• 웹기반 검사시스템 개발 및 유지보수\n\n• 신규 프로젝트 개발,"PHP 중급 이상\nMYSQL 중급 이상\nPHP 프레임워크 사용 경험자 (ci, ...",웹 서비스 라이브 런칭 경험자\nRESTful API 설계 및 개발경험\nGit H...,Git\nLinux\nMySQL\nPHP\nDocker\nSVN


In [10]:
# NaN값 존재 시 에러 발생하므로, Nan값을 " "으로 대체
df2 = df2.fillna(" ")
df2.head()

,직무명,주요업무,자격요건,우대사항,기술스택 ・ 툴
0,웹 개발자,• 세무 알고리즘 자동화 설계 및 개발\n• 세무팀과 커뮤니케이션\n• AWS 인프...,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\n• Python(필수), 웹...","• MSA 기반의 시스템 개발 경험이 있으신 분\n• AWS 를 활용한 개발, 운영...",Github\nJavaScript\nTypeScript\nReact.js\nVue.JS
1,웹 개발자,"• 세무 관련 서비스 및 플랫폼의 프론트엔드 에플리케이션 개발\n• 요구사항 분석,...",• HTML / CSS / JavaScript 에 대한 이해\n• 컴퓨터 사이언스에...,"• Vue.js, React.js, Angular 등 SPA 개발 경험이 있는 분\...",Github\nJavaScript\nTypeScript\nReact.js\nVue.JS
2,웹 개발자,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 프론트엔드 서비스 개발 및 유지 관리 경험을 가지신...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,
3,웹 개발자,[어떤 업무를 하게 되나요?] \n• 알트에디터 (영상 번역 | 자막생성 에디터 ...,• 3년 이상 또는 그에 준하는 백엔드 서비스 개발 및 유지 관리 경험을 가지신 분...,[이런 분은 더욱 환영합니다.]\n• 팀 리드 경험이 있으신 분\n• TTS / S...,
4,웹 개발자,• 웹기반 검사시스템 개발 및 유지보수\n\n• 신규 프로젝트 개발,"PHP 중급 이상\nMYSQL 중급 이상\nPHP 프레임워크 사용 경험자 (ci, ...",웹 서비스 라이브 런칭 경험자\nRESTful API 설계 및 개발경험\nGit H...,Git\nLinux\nMySQL\nPHP\nDocker\nSVN


In [11]:
# 숫자, 영어, 한글만 남기고 삭제 후 꼬꼬마 형태소분석 적용
def morph_analysis(row):
    result = []
    for column in row.index:
        text = str(row[column]).replace('\n', ' ')
        text = re.sub('[^0-9a-zA-Z가-힣 .,#]', '', text)
        tokens = kkma.pos(text)
        result.append(tokens)
    return result

start_time = time.time()  # 코드 실행 시작 시간 기록
df_result = df2.apply(morph_analysis, axis=1, result_type='expand')
df_result.columns = df2.columns

end_time = time.time()  # 코드 실행 종료 시간 기록
total_time = end_time - start_time  # 전체 코드 실행 시간 계산
print("Total Execution Time:", total_time, "seconds")
df_result.head()

java.lang.NullPointerException: java.lang.NullPointerException: Cannot invoke "java.util.List.size()" because "melAnalResult" is null

In [8]:
# morphs로 다시 뽑아봅니다
df1 = df.copy()
df2 = df1[:10]

# 숫자, 영어, 한글만 남기고 삭제 후 한나눔 형태소분석 적용
def morph_analysis(row):
    result = []
    for column in row.index:
        text = str(row[column]).replace('\n', ' ')
        text = re.sub('[^0-9a-zA-Z가-힣 .,]', '', text)
        tokens = hannanum.morphs(text)
        result.append(tokens)
    return result

start_time = time.time()  # 코드 실행 시작 시간 기록
df_result = df2.apply(morph_analysis, axis=1, result_type='expand')
df_result.columns = df2.columns

end_time = time.time()  # 코드 실행 종료 시간 기록
total_time = end_time - start_time  # 전체 코드 실행 시간 계산
print("Total Execution Time:", total_time, "seconds")
df_result.head()

Total Execution Time: 0.44213199615478516 seconds


,직무명,채용공고명,해시태그,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,근무지역,회사명,회사분야,출처
0,"[웹, 개발자]","[Python, Developer, 백엔드, 주니어]","[퇴사율5이하51300명설립49년육아휴직출산휴가스톡옵션건강검진교육비IT,, 컨텐츠]","[간단소개, 혜움랩스, 는, 세무, ,, 재무, ,, 경리, 등, AI, 기반, 비...","[세무, 알고리즘, 자동화, 설계, 및, 개발, 세무팀, 과, 커뮤니케이션, AWS...","[최소, 2년, 이상, 혹은, 그, 에, 준하, 는, 해당, 업무, 경험, Pyth...","[MSA, 기반, 의, 시스템, 개발, 경험, 이, 있, 으시ㄴ, 분, AWS, 를...","[몰입, 을, 위하, ㄴ, 액면, 가, 스톡옵션, 의, 기회, 가, 열리, 어, 있...","[Github, JavaScript, TypeScript, React, ., js,...","[서울, ., 한국]",[혜움랩스],"[IT, ,, 컨텐츠]","[httpswww, ., wanted, ., co, ., krwd167436]"
1,"[웹, 개발자]","[프론트엔드, 개발자, 주니어]","[퇴사율5이하51300명설립49년육아휴직출산휴가스톡옵션건강검진교육비IT,, 컨텐츠]","[간단소개, 혜움랩스, 는, 세무, ,, 재무, ,, 경리, 등, AI, 기반, 비...","[세무, 관련, 서비스, 및, 플랫폼, 의, 프론트엔드, 에플리케이션, 개발, 요구...","[HTML, CSS, JavaScript, 에, 어, 대하, ㄴ, 이해, 컴퓨터, ...","[Vue, ., js, ,, React, ., js, ,, Angular, 등, S...","[몰입, 을, 위하, ㄴ, 액면, 가, 스톡옵션, 의, 기회, 가, 열리, 어, 있...","[Github, JavaScript, TypeScript, React, ., js,...","[서울, ., 한국]",[혜움랩스],"[IT, ,, 컨텐츠]","[httpswww, ., wanted, ., co, ., krwd167433]"
2,"[웹, 개발자]","[프론트엔드, 개발자, Software, Engineer, 3년, 이상, 사본]","[출산휴가스타트업반려동물음료커피간식건강검진운동비IT,, 컨텐츠]","[유섭, 은, 현재, 번역, 서비스, Subup, 및, 자막, 편집, 에디터AltE...","[어떤, 업무, 를, 하, 게, 되, 나, 요, 알트에디터, 영상, 번역, 자막생성...","[3년, 이상, 또는, 그, 에, 준하, 는, 프론트엔드, 서비스, 개발, 및, 유...","[이런, 분, 은, 더욱, 환영, 하, ㅂ니다, ., 팀, 리드, 경험, 이, 있,...","[자유로운, 휴가, 무제한, 무, 급, 휴가, 동료, 와, 원활한, 협업, 을, 위...",[nan],"[서울, ., 한국]",[유섭],"[IT, ,, 컨텐츠]","[httpswww, ., wanted, ., co, ., krwd167134]"
3,"[웹, 개발자]","[백엔드, 개발자, Backend, Software, Engineer, 3년, 이상...","[출산휴가스타트업반려동물음료커피간식건강검진운동비IT,, 컨텐츠]","[유섭, 은, 현재, 번역, 서비스, Subup, 및, 자막, 편집, 에디터AltE...","[어떤, 업무, 를, 하, 게, 되, 나, 요, 알트에디터, 영상, 번역, 자막생성...","[3년, 이상, 또는, 그, 에, 준하, 는, 백엔드, 서비스, 개발, 및, 유지,...","[이런, 분, 은, 더욱, 환영, 하, ㅂ니다, ., 팀, 리드, 경험, 이, 있,...","[자유로운, 휴가, 무제한, 무, 급, 휴가, 동료, 와, 원활한, 협업, 을, 위...",[nan],"[서울, ., 한국]",[유섭],"[IT, ,, 컨텐츠]","[httpswww, ., wanted, ., co, ., krwd167133]"
4,"[웹, 개발자]","[웹개발, 인재PHP]","[인원급성장51300명설립10년이상반려동물음료사내카페복지포인트IT,, 컨텐츠]","[다음, 과, 같, 이, 유능, 하고, 역량, 있, 는, 인재, 를, 채용, 하, ...","[웹기반, 검사시스템, 개발, 및, 유지보수, 신규, 프로젝트, 개발]","[PHP, 중급, 이상, MYSQL, 중급, 이상, PHP, 프레임워크, 사용, 경...","[웹, 서비스, 라이브, 런칭, 경험자, RESTful, API, 설계, 및, 개발...","[매월, 1회, 패밀리데이오전근무세번째, 금요일, 패밀리데이, 상품권, 지급연, 6...","[Git, Linux, MySQL, PHP, Docker, SVN]","[경기, ., 한국]",[인투씨엔에스],"[IT, ,, 컨텐츠]","[httpswww, ., wanted, ., co, ., krwd166922]"


In [12]:
# '자격요건' 컬럼의 NaN 값을 빈 문자열로 대체
df['자격요건'].fillna('', inplace=True)

# '해시태그' 컬럼의 모든 단어를 하나의 리스트로 추출
all_skills = df['자격요건'].tolist()

# 단어 빈도 계산
word_counts = Counter([word for skills in all_skills for word in skills])

# 가장 많이 언급된 단어 추출 (상위 30개)
most_common_words = word_counts.most_common(30)

# 출력
for word, count in most_common_words:
    print(f"{word}: {count}")


 : 307293

: 37412
이: 31449
•: 27124
e: 25213
분: 20933
a: 20071
t: 20046
경: 19703
,: 18954
i: 18808
r: 17124
o: 16808
n: 16351
신: 14302
S: 13500
험: 13086
한: 12997
하: 12736
에: 12037
개: 11669
s: 11330
c: 11010
발: 10903
는: 10509
-: 10507
있: 10135
을: 9253
력: 9218
l: 9200
